# Interactive Time Series Visualization Assignment

In [1]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact

### Load the Online Retail.xlsx Excel file into a Pandas dataframe.

In [2]:
retail=pd.read_excel('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Time%20Series%20Analysis/Time%20Series%20-%20Day%202/Online%20Retail.xlsx')

In [7]:
retail.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


### Create an interactive histogram to explore the distributions of daily (by date) Quantity, UnitPrice, and Revenue. Include a dropdown widget where you can switch between the three fields.

**Hint:** You will need to create a Date field and then aggregate on it. The Quantity and Revenue fields should be summed and the UnitPrice field should be averaged (mean).

In [12]:
retail['Year']= retail.InvoiceDate.dt.year
retail['Month']= retail.InvoiceDate.dt.strftime('%B')
retail['Week']= retail.InvoiceDate.dt.isocalendar().week
retail['Weekday']= retail.InvoiceDate.dt.strftime('%A')
retail['Day']= retail.InvoiceDate.dt.day
retail['Date']= retail.InvoiceDate.dt.date
retail['Hour']= retail.InvoiceDate.dt.hour
retail['Minute']= retail.InvoiceDate.dt.minute

In [29]:
grouped_df = retail.groupby('Date', as_index=False).agg({'Quantity':'sum', 'Revenue':'sum','UnitPrice':'mean'})
#grouped_df

In [32]:
my_dropdown = ['Quantity', 'UnitPrice', 'Revenue']

@interact(Selection = my_dropdown)

def ihistogram(Selection, title='Distribution Of Metrics'):
    grouped_df = retail.groupby('Date', as_index=False).agg({'Quantity':'sum', 'Revenue':'sum','UnitPrice':'mean'})
    
    fig = px.histogram(grouped_df, 
                       x=Selection, 
                       title=title,
                      template='none').update(layout=dict(title=dict(x=0.5)))
    fig.update_yaxes(title_text='Number of Records')
    fig.update_traces(marker_color='lightskyblue',
                  marker_line_color='black',
                  marker_line_width=1)
    fig.show()

interactive(children=(Dropdown(description='Selection', options=('Quantity', 'UnitPrice', 'Revenue'), value='Q…

### Create an interactive line chart where you can explore Quantity and Revenue by day (date) for each country. Include two dropdown widgets - one to choose between the Quantity and Revenue fields and the other to choose the Country.

In [40]:
#np.sort(retail.Country.unique())

In [ ]:
@interact(field=['Quantity', 'Revenue'], country=np.sort(df['Country'].unique()))

def daily_line_country(field, country):
    agg_country = retail.loc[df['Country']==country].groupby(['Date'], as_index=False).agg({'Quantity':'sum', 'Revenue':'sum'})
    fig = px.line(agg_country, 'Date', field, title=f'Plot of daily {field} in {country}', template='none')
    fig.show()

In [52]:

metrics = ['Quantity', 'Revenue']
countries = retail.Country.unique()

@interact(Metric=metrics, Country=countries)

def plot_line_chart(Metric, Country):
    df_filtered = retail.loc[lambda x: x.Country == Country]
    df_agg = df_filtered.groupby('Date').sum()
    fig = px.line(df_agg, x=df_agg.index, y=Metric)
    fig.show()

interactive(children=(Dropdown(description='Metric', options=('Quantity', 'Revenue'), value='Quantity'), Dropd…

In [69]:
metrics = ['Quantity', 'Revenue']
countries = np.sort(retail.Country.unique())

@interact(metric_field=metrics, country_field=countries)

def plot_line_chart(metric_field, country_field, Title='Distribution of Metrics by Country'):
    country_df = retail.loc[retail.Country == country_field].groupby('Date', as_index=False).sum()
    
    fig = px.line(country_df, 
                  x= country_df.index, 
                  y= metric_field,
                  title=Title,
                  template='none').update(layout=dict(title=dict(x=0.5))
                                         )
    fig.update_yaxes(title_text='Number of Records')
    fig.update_traces(marker_color='lightskyblue',
                  marker_line_color='black',
                  marker_line_width=1)
    fig.show()

interactive(children=(Dropdown(description='metric_field', options=('Quantity', 'Revenue'), value='Quantity'),…

### Create an interactive scatter plot showing the relationships between daily Quantity, UnitPrice, and Revenue for the United Kingdom. Include two dropdown boxes that let you choose between the 3 fields - one for the x axis and one for the y axis of your scatter plot. Size the data points according to Revenue.

In [71]:
#retail.loc[retail.Country == 'United Kingdom'].groupby('Date', as_index=False).agg({'Quantity':'sum', 'Revenue':'sum'})

In [75]:
x_metrics = ['Quantity', 'UnitPrice', 'Revenue']
y_metrics = ['Quantity', 'UnitPrice', 'Revenue']

@interact(x_axis=x_metrics, y_axis=y_metrics)

def scatter_plot(x_axis, y_axis, Title='Distribution of Metrics by Country'):
    uk_df = retail.loc[retail.Country == 'United Kingdom'].groupby('Date', as_index=False).agg({'Quantity':'sum', 'Revenue':'sum', 'UnitPrice':'mean'})

    fig = px.scatter(uk_df, 
                     x= x_axis, 
                     y= y_axis, 
                     size= 'Revenue', 
                     title=Title, 
                     template='none').update(layout=dict(title=dict(x=0.5))
                                            )
    fig.update_yaxes(title_text='Number of Records')
    fig.update_traces(marker_color='lightskyblue',
    marker_line_color='black',
    marker_line_width=1)
    fig.show()

interactive(children=(Dropdown(description='x_axis', options=('Quantity', 'UnitPrice', 'Revenue'), value='Quan…

### Create a bar chart showing the top X products sold (by quantity) in the United Kingdom in a specific month. Use a dropdown box to select the month name and a slider to show the top X products. The range for X should be from 5 to 25. Make sure the bars are sorted in descending order according to their total quantity sold for the month.

Bonus points if you can figure out how to ensure that the month names in the dropdown appear in their correct order (January, February, March, April, etc.).

In [89]:
field = [
    'January', 
    'February', 
    'March', 
    'April', 
    'May', 
    'June', 
    'July', 
    'August', 
    'September', 
    'October', 
    'November', 
    'December'
]

@interact(top_products=(5,25), month=field)

def plot_bar_chart(top_products, month, Title='Top Product Sold by Month in UK'):
    uk_monthly_df = retail.loc[
        retail.Country == 'United Kingdom'
    ].groupby('Description').sum().sort_values('Quantity', ascending=False).head(top_products)
    
    fig = px.bar(uk_monthly_df, 
                     x= 'Quantity', 
                     color= uk_monthly_df.index,
                     title=Title, 
                     template='none').update(layout=dict(title=dict(x=0.5))
                                            )
    fig.update_yaxes(title_text='Number of Records')
    fig.update_traces(marker_color='lightskyblue',
    marker_line_color='black',
    marker_line_width=1)
    fig.show()

interactive(children=(IntSlider(value=15, description='top_products', max=25, min=5), Dropdown(description='mo…